# Creando una REST API con Flask y sus amigos

Bienvenidos a este nuevo tutorial (taller) impartido por la comunidad Python Nicaragua.

En este taller construiremos una aplicación que "emule" a https://www.meetup.com

Para eso usaremos el popular microframework de Python Flask y usaremos algunas otras librerías que nos ayudaran a construir una API mas o menos completa.

El taller estará divido en 3 fases:

1. Construiremos una API usando Flask y Python puro
2. Construiremos una API usando Flask y SQLAlchemy
3. Construiremos una API usando Flask, SQLAlchemy y Connexion

El repositorio con todo el proyecto finalizado lo puedes encontrar aquí https://github.com/darwing1210/flask-meetup

## Introducción

### Qué es una REST API?

Una API es un conjunto de procesos, métodos y subrutinas que cumplen una o muchas funciones con el fin de ser utilizadas por otro software (estas siglas provienen del inglés y son el acrónimo de “Application Programming Interface”, en español: “Interfaz de Programación de Aplicaciones”).

REST o Representational State Transfer es un ESTILO  de Arquitectura a la hora de realizar una comunicación entre cliente y servidor, que aprovechan el protocolo HTTP para proporcionar un comportamiento CRUD (Crear, Leer, Actualizar y Eliminar) en objetos y colecciones de esas objetos. El comportamiento CRUD se correlaciona muy bien con los verbos del método del protocolo HTTP como este:

|Action|HTTP Verb|Description|
|--- |--- |--- |
|Create|POST|Crea un nuevo objeto|
|Read|GET|Lee la información de un objeto o una colección de objetos|
|Update|PUT|Actualiza la información de un objeto existente|
|Delete|DELETE|Elimina un objeto|


### Qué es Flask?

Flask es un microframework para crear aplicaciones web con Python. Está diseñado para que hacer aplicaciones sea rápido y fácil pero sin perder la capacidad de escalar a aplicaciones complejas.

Se diferencia de Django y de otros frameworks porque no trae baterías incluídas, es decir viene con los utilidades y bibliotecas básicas para ejecutar una aplicación web, lo que hace que sea el desarrollador quien decida como hacer la arquitectura de su aplicación.

## **Manos a la obra...**
### Qué necesitas?

- Conocimiento básico de Python
- Saber usar la consola de tu sistema operativo
- Python 3.4+


### Instalando Python

Para Instalar Python puedes descargar la ultima versión (3.8) de https://www.python.org/downloads
    
Puedes seguir el tutorial de las DjangoGirls de como instalar Python en tu SO https://tutorial.djangogirls.org/es/python_installation/

Verifica tu versión de Python ejecutando:

```console
$ python --version
```

Una vez que tengas Python instalado, crearemos un nuevo directorio y nos movemos a el:

```console
$ mkdir flask-meetup
$ cd flask-meetup
```

Crearemos un nuevo entorno virtual:

```console
$ python -m venv venv
```

Activaremos el entorno virtual:

```console
$ source venv/bin/activate
```

en Windows como administrador:

```console
$ venv\Scripts\activate
```

Listo... una vez configurado el entorno virtual procederemos a instalar Flask usando el comando pip

```console
$ pip install Flask
// o
$ python -m pip install Flask 
```

### Creando una aplicación básica

1. Crea un nuevo archivo, nombralo `hola.py`
2. Agrega el siguiente código

```hola.py
'''
Esta es la aplicación mas sencilla que puedes hacer con Flask 
Tomado y traducido de https://palletsprojects.com/p/flask/
'''

from flask import Flask, escape, request

app = Flask(__name__)

@app.route('/')
def hola():
    nombre = request.args.get('nombre', 'mundo')
    return f'Hola {escape(nombre)}!'
```

Ve a tu consola, antes de ejecutar la aplicación, necesitaremos definir una variable de entorno, para hacerlo, ejecuta:

```console
$ export FLASK_APP=hola.py
```

*En Windows ejecuta `set` en lugar de `export`

Ahora que tenemos todo listo, ejecutemos nuestra aplicación:

```console
$ flask run
* Serving Flask app "microblog"
* Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
```

Abre en tu navegador http://127.0.0.1:5000/?nombre=Luis

**Listo!!!** hemos concluido el taller... bromeo, esa es tu primera aplicación en Flask, así de sencillo

#### Tip

Evitemos tener que cargar las variables de entorno cada vez que cargamos nuestra terminal, una alternativa es usar **python-dotenv** el cual cargara todas nuestras variables definidas en un archivo `.env` o `.flaskenv`.

Para ello ejecuta:

```console
$ pip install python-dotenv
```

y agrega tus variables de entorno al archivo `.flaskenv`

```.flaskenv
# Agrega aquí las variables de entorno
FLASK_APP=hola.py
```

## Parte 1: Aplicación usando Python puro

Crearemos un nuevo módulo (una nueva app), para ello crearemos un nuevo folder, lo llamaremos `meetup_app`
y dentro de el, agregaremos un archivo `__init__.py`, en este archivo crearemos una función para inicializar
nuestra app.

el código inicial debería lucir así:
    
```__init__.py
from flask import Flask


def crear_app(configuracion):
    app = Flask(__name__)

    return app

```

Necesitamos agregar algunas configuraciones para nuestro proyecto, esto lo podemos hacer ya sea con una clase
o con un diccionario, en nuestro caso usaremos una clase definida en un nuevo archivo `config.py`

```config.py
# Configuraciones de nuestra aplicación
import os

class Config(object):
    '''Configuración principal.'''
    DEBUG = False
    CSRF_ENABLED = True
    SQLALCHEMY_DATABASE_URI = os.getenv('DATABASE_URL')


class DevConfig(Config):
    '''Configuraciones para desarrollo.'''
    DEBUG = True


app_config = {
    'development': DevConfig,
    # 'testing': TestingConfig,
    # 'staging': StagingConfig,
    # 'production': ProductionConfig,
}
```

Para cargar estas configuraciones en nuestra app, necesitamos importar el mapa de configuraciones en `__init__.py`

```
from meetup_app.config import app_config
```

y cargarlas en nuestra app dentro de la función `crear_app`, usando:

```
app.config.from_object(app_config[configuracion])
```

El archivo final luce así:

```__init__.py
# Inicializando configuración
from meetup_app.config import app_config

from flask import Flask


def crear_app(configuracion):
    app = Flask(__name__)
    app.config.from_object(app_config[configuracion])

    return app

```

Una vez cargadas las configuraciones, necesitamos ejecutar la aplicación, pero no lo háremos desde este modulo,
sino que crearemos uno llamado `run.py` el cual cargara la aplicación creando una nueva instancia de esta.
El archivo luce así: 

```run.py
import os

from meetup_app import crear_app

configuracion = os.getenv('FLASK_ENV') # configuracion = 'development'

app = crear_app(configuracion)

if __name__ == '__main__':
    app.run()
```

Para ejecutar recuerda o definir las variables de entorno de la siguiente manera o crear un archivo 
de variables de entorno `.flaskenv`:
    
```.flaskenv
# Agrega aquí las variables de entorno
FLASK_APP=run.py
FLASK_ENV=development
```

Ahora ejecuta el app con `flask run` notaras un par de cosas interesantes:

```console
$ flask run
 * Serving Flask app "run.py" (lazy loading)
 * Environment: development
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 202-965-352
```

El app corre desde las configuraciones de desarrollo y el modo debug esta activado, lo cual nos ayudara a entender mejor los errores cuando estos aparezcan. (y también tienes autoreload)

### Agregando controladores/vistas

Pero que sucede?, nuestra app no hace nada aun, para ello debemos registrar urls y controladores.

Agregaremos un nuevo modulo llamado `controllers.py` en el que registraremos nuestras urls y definiremos los controladores.

Para poder registrar urls fuera de el módulo principal, necesitamos definir un `Blueprint` el cual no es mas que una guía para que el app pueda traducir urls basadas en el path y métodos aceptados por la url.

Debemos importarlo así `from flask import Blueprint` y luego creamos una instancia de `Blueprint`, le llamaremos `urls`

```.py
from flask import Blueprint
urls = Blueprint('urls', __name__)
```

Una vez creada la instancia del Blueprint urls podremos registrar nuestros controladores de la siguiente manera:

```.py
from flask import escape, request
...
@urls.route('/')
def hola():
    nombre = request.args.get('nombre', 'mundo')
    return f'Hola {escape(nombre)}!'
```

### Cargando una plantilla

Con Flask podemos renderizar una plantilla HTML usando Jinja2, para ello requerimos crear una carpeta llamada `templates/` e incluir los archivos HTML, en esta caso crearemos uno llamado `bienvenidos.html` y agregaremos el siguiente contenido:

```bienvenidos.html
<!DOCTYPE html>
<html>
<head>
	<title>Bienvenidos</title>
</head>
<body style="background-color: black; text-align: center">
	<h1 style="color: white">Hola a todos, Bienvenidos al taller de Flask</h1>
</body>
</html>
```

Necesitmos definir un nuevo controlador y una nueva url para cargar el contenido:

```.py
from flask import render_template
...
@urls.route('/bienvenidos')
def bienvenidos():
    return render_template('bienvenidos.html')
```

### TODO

- Explicar cada método dentro de controllers.py

## Parte 2: Usando una Base de datos y SQLAlchemy

### TODO

- Explicar como instalar flask-sqlalchemy y flask-migrate (para que sirven)
- Actualizar `__init__.py` con las extenciones
- Inicializar Flask Migrate `flask db init`
- Crear archivo de modelos, crear modelo User
- Importar modelos en `__init__.py`
- Generar migración de User
- Aplicar migración
- Crar modelo Evento
- Generar y Aplicar migración de Evento
- Explicar relación many to many y tabla intermedia
- Explicar el to_dict() de los modelos
- Actualizar controladores para que usen los modelos Evento
- Evitar dolores de cabeza con fechas instalando `pip install python-dateutil`
- Ver BD en https://sqlitebrowser.org/dl/
- Crear controlador para registrar usuario
- Instalar flask-httpauth `$ pip install flask-httpauth`
- Configurar HTTPBasicAuth para los endpoints de registrarse a eventos
- Explicar el método `verify_password`
- Explicar que es g
- Cargar `auth` en controllers
- Crear controladores para unirse/salirse de evento
- Explicar la necesidad de connexion (parsers, validators, etc)
- Explicar como configurar app con conexion
- Crear swagger.yml, primero Definiciones, luego rutas
- Actualizar Controllers para que usen los parametros de connexion